## Importing the Required Libraries

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import io
import cv2

# Only enable this for tensor-core GPUs
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# Hyperparameters

In [2]:
RESIZE_TO = 28
PATCH_SIZE = 28

NUM_MIXER_LAYERS = 2
HIDDEN_SIZE = 64
MLP_SEQ_DIM = 64
MLP_CHANNEL_DIM = 64

EPOCHS = 5
BATCH_SIZE = 128

# Dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
x_train[0].shape

(28, 28)

In [5]:
X_train=[]
X_test=[]

In [6]:
for i in x_train:
    X_train.append(cv2.cvtColor(i, cv2.COLOR_GRAY2RGB))

In [7]:
for i in x_test:
    X_test.append(cv2.cvtColor(i, cv2.COLOR_GRAY2RGB))

In [8]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [9]:
X_train.shape

(60000, 28, 28, 3)

## MLP-Mixer Utilities

In [10]:
def mlp_block(x, mlp_dim):
    x = layers.Dense(mlp_dim)(x)
    x = tf.nn.gelu(x)
    return layers.Dense(x.shape[-1])(x)

def mixer_block(x, tokens_mlp_dim, channels_mlp_dim):
    y = layers.LayerNormalization()(x)
    y = layers.Permute((2, 1))(y)
    
    token_mixing = mlp_block(y, tokens_mlp_dim)
    token_mixing = layers.Permute((2, 1))(token_mixing)
    x = layers.Add()([x, token_mixing])
    
    y = layers.LayerNormalization()(x)
    channel_mixing = mlp_block(y, channels_mlp_dim)
    output = layers.Add()([x, channel_mixing])
    return output

def mlp_mixer(x, num_blocks, patch_size, hidden_dim, 
              tokens_mlp_dim, channels_mlp_dim,
              num_classes=10):
    x = layers.Conv2D(hidden_dim, kernel_size=patch_size,
                      strides=patch_size, padding="valid")(x)
    x = layers.Reshape((x.shape[1]*x.shape[2], x.shape[3]))(x)

    for _ in range(num_blocks):
        x = mixer_block(x, tokens_mlp_dim, channels_mlp_dim)
    
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.GlobalAveragePooling1D()(x)
    return layers.Dense(num_classes, activation="softmax", dtype="float32")(x)

In [11]:
def create_mlp_mixer():
    inputs = layers.Input(shape=(28, 28, 3))
    outputs = mlp_mixer(inputs, NUM_MIXER_LAYERS,
                        PATCH_SIZE, HIDDEN_SIZE, 
                        MLP_SEQ_DIM, MLP_CHANNEL_DIM)
    return tf.keras.Model(inputs, outputs, name="mlp_mixer")

In [12]:
mlp_mixer_classifier = create_mlp_mixer()

In [13]:
def run_experiment(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )
    
    
    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, top_1_accuracy = model.evaluate(X_test, y_test)
    print()
    print(f"Test accuracy: {round(top_1_accuracy * 100, 2)}%")
    
    return history, model

## Model Training and Evaluation

In [14]:
history, model = run_experiment(mlp_mixer_classifier)
model.save(f"mlp_mixer_{NUM_MIXER_LAYERS}")

Epoch 1/5
422/422 [==============================] - 16s 17ms/step - loss: 0.6246 - accuracy: 0.8248 - val_loss: 0.2439 - val_accuracy: 0.9295
Epoch 2/5
422/422 [==============================] - 6s 14ms/step - loss: 0.2783 - accuracy: 0.9170 - val_loss: 0.1670 - val_accuracy: 0.9517
Epoch 3/5
422/422 [==============================] - 6s 13ms/step - loss: 0.1786 - accuracy: 0.9477 - val_loss: 0.1215 - val_accuracy: 0.9648
Epoch 4/5
422/422 [==============================] - 6s 14ms/step - loss: 0.1138 - accuracy: 0.9656 - val_loss: 0.1013 - val_accuracy: 0.9677
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.1013 - accuracy: 0.9699

Test accuracy: 96.99%
